<table style="width: 100%;"> <tr> <td style="width: 20%; vertical-align: top;"> <img src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" alt="UdeA" height="150px"> </td> <td style="width: 80%; padding-left: 20px;"> <strong style="font-size: 30px;">EXPLORACIÓN DEL CONJUNTO DE DATOS PARA EL DESARROLLO DE UN SISTEMA DE RECOMENDACIÓN DE PELÍCULAS - PROYECTO MARKETING</strong><br><br> <span style="font-size: 30px;"> Luisa Fernanda Alzate Cuartas <br> Juan Camilo Henao Caro<br>Isabella Mendez Hoyos<br> Fernando Antonio Piñeres Ramos </span> </td> </tr> </table>

#### LIBRERÍAS REQUERIDAS

In [1]:
import pandas as pd # Manipulación y el analísis de datos.
import sqlite3 as sql # Interactuar con base de datos SQL usando Python.

#### CONEXIÓN CON LA BASE DE DATOS

In [5]:
#Establece una conexión a la base de datos SQLite ubicada en "content/db_movies"
conexion = sql.connect("/content/db_movies")

# Crear un cursor a partir de la conexión, que permita ejecutar comando SQL sobre la base de datos.
cur = conexion.cursor()

In [6]:
# Ejecuta una consulta SQL que obtiene los nombres de todas las tablas existentes en la base de datos.
cur.execute("SELECT name FROM sqlite_master where type='table' ")

# Recupera todos los resultados de la consulta anterior en forma de lista de tuplas.
cur.fetchall()

[]

#### EXPLORACIÓN DE LAS TABLAS

##### TABLA MOVIES

In [7]:
# Ejecuta la consulta SQL "SELECT * FROM movies" sobre la base de datos y carga los resultados en un DataFrame de pandas.
pd. read_sql_query("SELECT * FROM movies", conexion)

DatabaseError: Execution failed on sql 'SELECT * FROM movies': no such table: movies

Transformaciones necesarias para la tabla movies: separar de la columna title el nombre de la película de la fecha de estreno y separar los generos.

In [ ]:
pd.read_sql_query("""
select * from movies where title like '%Goodnight Mommy%'

""", conexion)


,movieId,title,genres
0,139655,Goodnight Mommy (Ich seh ich seh) (2014),Drama|Fantasy|Horror|Thriller


Alguna filas tienendos títulos entre paréntesis (porque puede estar el título original y la traducción).

Entonces, debemos hacer dos separaciones:

- Primero detectar si hay más de un par de paréntesis.

- Luego extraer el año, que siempre es el último paréntesis.

In [ ]:
#Separación el nombre de la pelicula y el año de la tabla movies usando sql
pd.read_sql_query("""
    SELECT
        SUBSTR(title, 1, INSTR(title, '(') - 2) AS movie_title,
        SUBSTR(title, INSTR(title, '(') + 1, INSTR(title, ')') - INSTR(title, '(') - 1) AS release_year
    FROM movies
""", conexion)


,movie_title,release_year
0,Toy Story,1995
1,Jumanji,1995
2,Grumpier Old Men,1995
3,Waiting to Exhale,1995
4,Father of the Bride Part II,1995
...,...,...
9737,Black Butler: Book of the Atlantic,2017
9738,No Game No Life: Zero,2017
9739,Flint,2017
9740,Bungo Stray Dogs: Dead Apple,2018


##### TABLA RATINGS

In [ ]:
# Ejecuta la consulta SQL "SELECT * FROM ratings" sobre la base de datos y carga los resultados en un DataFrame de pandas.
pd.read_sql_query("SELECT * FROM ratings", conexion)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


Transformaciones necesarias para la tabla ratings: Transformar la columna timestamp en formato fecha.

In [ ]:
# Consulta para obtener la tabla ratings con la columna timestamp convertida a fecha
ratings_df = pd.read_sql_query("SELECT *, datetime(timestamp, 'unixepoch') AS date FROM ratings", conexion)

# Muestra el DataFrame con la nueva columna de fecha
print(ratings_df.head())

ratings_df.to_sql("ratings_with_date", conexion, if_exists="replace", index=False)


   userId  movieId  rating  timestamp                 date
0       1        1     4.0  964982703  2000-07-30 18:45:03
1       1        3     4.0  964981247  2000-07-30 18:20:47
2       1        6     4.0  964982224  2000-07-30 18:37:04
3       1       47     5.0  964983815  2000-07-30 19:03:35
4       1       50     5.0  964982931  2000-07-30 18:48:51


100836

#### PREGUNTAS PARA EXPLORACIÓN GENERAL

¿Cuántas películas hay registradas en total?

¿Cuántos géneros diferentes existen?

¿Cuál es el género más común entre las películas?

¿Cuántas calificaciones (ratings) hay en total?

¿Cuántos usuarios distintos han dado calificaciones?

¿Qué película tiene más calificaciones?

¿Qué usuario ha dado más calificaciones?

¿Cuál es la distribución de las calificaciones? (¿hay más 5.0, 4.0, etc.?)

¿Existen películas sin ninguna calificación?

¿Cuál es el promedio de calificación por película?

¿Cuál es el promedio de calificación por usuario?

¿Cuál es el rango de años de las películas? (¿de qué año a qué año?)

#### PREGUNTAS PARA DETENCCIÓN DE PROBLEMAS O CALIDAD DE DATOS

¿Hay películas con el mismo nombre pero diferente movieId?

¿Hay calificaciones fuera del rango esperado (por ejemplo, menores a 0 o mayores a 5)?

¿Existen valores nulos o vacíos en alguna columna?

¿Hay géneros desconocidos o no estandarizados?

¿Hay películas sin año de estreno?

#### PREGUNTAS PARA VISUALIZACIÓN DE DISTRIBUCIONES

Cómo es la distribución de las calificaciones?

¿Cómo es la distribución del número de calificaciones por película?

¿Cómo es la distribución del número de calificaciones por usuario?

¿Cómo varía la calificación promedio a lo largo de los años?

#### PREGUNTA ANÁLISIS AVANZADO



(Cruzar tablas y buscar patrones)

¿Cuál es el género que tiene mejores calificaciones promedio?

¿Las películas más recientes tienen mejores calificaciones que las antiguas?

¿Qué usuarios tienden a calificar más bajo o más alto?

¿Hay algún patrón entre género y año de estreno?

¿Cuál es la película mejor calificada?

¿Cuál es la película peor calificada?

¿Existe algún año donde se estrenaron muchas películas de un género específico?

¿Cómo cambia la popularidad de géneros a lo largo del tiempo?